In [ ]:
import numpy as np

import arviz as az

from matplotlib import pyplot as plt

from scipy.interpolate import BSpline

from pymagglobal.utils import i2lm_l, i2lm_m

from common import radData, gamma_0, mean_solar

from radio_pyMC import calc_HPA, prod_Be10, prod_C14

In [ ]:
prefix = 'radio_fdp2d'
idata_fname = f'../dat/{prefix}_result.nc'

with np.load(f'../dat/{prefix}_ensemble.npz') as fh:
    knots = fh['knots']
    coeffs = fh['coeffs']
    knots_solar = fh['knots_solar']
    solar = fh['solar']

iData = az.from_netcdf(idata_fname)

In [ ]:
ind = 0
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

ax.plot(
    knots,
    coeffs[ind, :, ::2],
    alpha=0.01,
    color="grey",
    zorder=0,
)
ax.plot(
    knots,
    np.mean(coeffs[ind, :, ::2], axis=1),
    color="C0",
    zorder=1,
)

ax.set_title(rf"$g_{i2lm_l(ind)}^{{{i2lm_m(ind)}}}$");

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

fac = 0.63712**3

dips = fac * np.sqrt(np.sum(coeffs[:3, :, ::1]**2, axis=0))

ax.plot(
    knots,
    dips,
    alpha=0.01,
    color="grey",
    zorder=0,
)
ax.plot(
    knots,
    np.mean(dips, axis=1),
    color="C0",
    zorder=1,
);

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

ax.plot(
    knots_solar,
    solar,
    alpha=0.01,
    color="grey",
    zorder=0,
)
ax.plot(
    knots_solar,
    np.mean(solar, axis=1),
    color="C0",
    zorder=1,
);

In [ ]:
interp_solar = BSpline(
    knots_solar,
    solar,
    1,
)
solar_at_g = interp_solar(knots)

In [ ]:
qGL = prod_C14(dips, solar_at_g)

In [ ]:
s_fac = np.array(iData.posterior['s_fac'])
s_fac = np.mean(s_fac.reshape(-1, 3), axis=0)

In [ ]:
q_GL_cal = prod_C14(fac*abs(gamma_0), mean_solar)
# XXX Parametrize the 0.05
cal_facs = q_GL_cal * (1 + s_fac*0.1)

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(
    knots,
    qGL[:, ::20],
    color='C0',
    alpha=0.1,
);
ax.plot(
    knots,
    np.mean(qGL, axis=1),
    color='C0',
);
ax.errorbar(
    radData['t'],
    radData['C14'] * cal_facs[2],
    color='grey',
    marker='.',
    yerr=0.05 * cal_facs[2],
    ls='',
)
ax.set_xlim(-2000, 0)
